## Tutorial 13. Data wrangling: Reproducing the y-ome paper 

Created by Emanuel Flores-Bautista 2019  All content contained in this notebook is licensed under a [Creative Commons License 4.0 BY NC](https://creativecommons.org/licenses/by-nc/4.0/). The code is licensed under a [MIT license](https://opensource.org/licenses/MIT).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib as mpl
import numba
import TCD19_utils as TCD

TCD.set_plotting_style_2()

%matplotlib inline
# This enables high res graphics inline
%config InlineBackend.figure_format = 'svg'

This tutorial is based on the [Ghatak *et al.* paper ](https://academic.oup.com/nar/article/47/5/2446/5304327) titled "The y-ome defines the 35% of Escherichia coli genes that lack experimental evidence of function". The authors have [posted all of the code for the analysis on Github](https://github.com/zakandrewking/y-ome) and is a perfect example of reproducible research. It even has a [binder container](https://mybinder.org/v2/gh/zakandrewking/y-ome/master?urlpath=lab/tree/notebooks) to reproduce the code on the cloud.  

Their goal was to get a better understanding of the genes in the *E.coli* genome that haven't been experimentally tested and thus lack annotation. These genes are very important because they have been kept in evolutionary time because most likely because they generate a function in cell phenotype. As the authors note, these argument is evident when thinking of [syn3.0, the third version of a synthetic bacterial cell](https://www.jcvi.org/first-minimal-synthetic-bacterial-cell) created in the John Craig Venter institute. This minimal cell contains only essential genes, of which more than 20% lack functional annotation with current computational methods.

They found that the genes that lack experimental evidence (the "y-ome") have on average, lower expression levels and are enriched in the termination region of the *E. coli* chromosome. The latter observation is related to the nature of DNA replication in *E. coli* but that's another story. 

In this tutorial, we will assess if the first finding of the authors extend to the experimental and hypothetical TFs of *E. coli*, using the data from a paper I worked on during my thesis. 

The workflow is as follows: 

> ###### We will extract the data from the y-ome paper and add the annotation for the TFs using pandas, and then, we will visualize the results using Seaborn. 

### Load y-ome data

In [ ]:
path = '../data/'

#Load proteomics dataset
prot= pd.read_csv(path +'proteomics.csv')

#Load RNA-seq dataset
trans = pd.read_csv(path +'tpm-log.tsv', sep = '\t')

#Load yome genes
yome_genes= pd.read_csv(path +'yome-genes.csv')

yome_genes = yome_genes[['locus_tag', 'annotation_quality']]

In [ ]:
yome_genes.head()

In [ ]:
trans.tail(3)

### Calculate mean log(TPM) values for all of the genes in the Ghatak dataset.

Because genes are in the rows of the dataset, we will have to transpose the dataframe and then apply the mean method, which computes the sample mean of all the columns in a dataframe. 

In [ ]:
#Transpose RNAseq dataset
trans_T = trans.iloc[:, 1:].T

In [ ]:
trans_T.head()

In [ ]:
#Get mean values for all the genes
mean = trans_T.mean()

In [ ]:
#Make a new dataframe for visualization
tpm_log_mean = pd.DataFrame({'mean_log_tpm': mean, 'locus_tag': trans.locus_tag})

In [ ]:
tpm_log_mean.head()

In order to visualize the $log(TPM)$ distribution of the *E.coli* by annotation, we need to merge the `yome_genes`dataframe with `tpm_log_mean`. In tutorial 2, we learned that we can do this operation with the `pd.merge` function. 

In [ ]:
tpm_log_yome = pd.merge(tpm_log_mean, yome_genes, on = "locus_tag")

In [ ]:
tpm_log_yome.shape

In [ ]:
tpm_log_yome.head(3)

In [ ]:
#Rename column for plotting
tpm_log_yome = tpm_log_yome.rename(columns= {'mean_log_tpm': '$log_{10}(TPM)$'})

In [ ]:
tpm_log_yome.head()

Now we are ready to visualize the results from the Ghatak paper. Let's visualize the distributions using a violinplot. 

In [ ]:
sns.violinplot(data = tpm_log_yome, 
              x = '$log_{10}(TPM)$', 
              y = 'annotation_quality', inner = 'quartile', palette = 'Greens_r');

Just for the reference, this is the plot from their paper. Notice that they are using ECDFs ! 

In [ ]:
from IPython.display import Image

Image(url= "https://raw.githubusercontent.com/eflobau/TCD_19/master/data/yome_exp_distro.png")

Awesome! Indeed, we can see that the median of the log(TPM) distribution of the genes with high quality annotation is nearly two orders of magnitude bigger than those with low annotation (y-ome)! Now we can go ahead and test if the same observation holds for transcription factors only. 

### Load *E. coli* gene names and locus tags

First, let's load the *E. coli* gene names and locus tags.

In [ ]:
ecoli_gene_names = pd.read_csv(path + 'ecoli_genes_locus.csv')

ecoli_gene_names.head()

### Load TF gene names from RegulonDB

Now we can go ahead and load the TF gene names. This dataset was downloaded from [RegulonDB](http://regulondb.ccg.unam.mx/menu/download/datasets/index.jsp). 

In [ ]:
col_names = ['ID','TF', 'gene_name', 'act_conf', 'inact_conf',
             'evidence', 'PMID', 'confidence']

df_TFs = pd.read_csv(path + 'TFSet.txt', comment = '#', delimiter= '\t', 
                  names = col_names)

In [ ]:
df_TFs.head(2)

Great! We now have to extract the gene names, and make them lower case in order to make them readily comparable to the Ghatak et al. datasets.

In [ ]:
#Extract TF gene names and make them lower case
tf_genes_names = df_TFs['gene_name'].values

tf_genes_names_l = [str(x).lower() for x in tf_genes_names]

In [ ]:
len(tf_genes_names_l)

Notice that this dataset only contains 215 TFs. 

### Load Pérez-Rueda lab hypothetical TF list

Now, we are going to load the hypothetical TFs found by my lab.

In [ ]:
#Load Pérez-Rueda lab hyp TF list 

hyp_tf_genes = pd.read_csv(path + 'hypTF_list_genes.csv')

# Extract hyp TF gene names as a numpy array
hyp_tf_gene_names = hyp_tf_genes['hyptfs'].values

hyp_tf_gene_names.shape

### Extract hypothetical and experimental TF locus tags

In [ ]:
#Extract experimental TFs list 
#Experimental TFs will be those genes annotated by RegulonDB
#that do not appear in the hypTF list from the Perez Rueda Lab

TFs_annot = []

for gene in ecoli_gene_names['gene_name']:
    if gene.lower() in tf_genes_names_l and gene not in hyp_tf_gene_names:
        TFs_annot.append('exp')
    
    elif gene.lower() in hyp_tf_gene_names:
        
        TFs_annot.append('hyp')
        
    else:
        TFs_annot.append('non_TF_protein')
                 

ecoli_gene_names['annot'] = TFs_annot

In [ ]:
ecoli_gene_names.tail()

In [ ]:
hyp = ecoli_gene_names[ecoli_gene_names['annot'] == 'hyp']

exp = ecoli_gene_names[ecoli_gene_names['annot'] == 'exp']

nonTF = ecoli_gene_names[ecoli_gene_names['annot'] == 'non_TF_protein']

#Extract TFs locus tags for each group
hyp_locus_tags =  hyp['locus_tag'].values

exp_locus_tags =  exp['locus_tag'].values

### Add TF annotation to the Ghatak *et al.* datasets

We now just have to add the TF annotation to the `tpm_log_yome`dataframe and we can visualize the results.

In [ ]:
tf_annot = []

for row in tpm_log_yome['locus_tag']:
    
    if row in exp_locus_tags :
        tf_annot.append('exp')
    elif row in hyp_locus_tags:
        
        tf_annot.append('hyp')
        
    else:
        tf_annot.append('non_TF') 
        
tpm_log_yome['TF_annotation'] = tf_annot

In [ ]:
tpm_log_yome.tail()

Finally, we can go ahead and plot the $log(TPM)$ distributions with the TF annotation. 

In [ ]:
ch = sns.cubehelix_palette(n_colors = 3, reverse = True)

In [ ]:
sns.violinplot(data = tpm_log_yome, y = '$log_{10}(TPM)$', x ='TF_annotation', 
              palette = ch, inner = 'quartile');

cool! Indeed, transcription factors with low quality annotation or hypothetical TFs, have lower expression levels than those with experimental validation. Another interesting thing to notice is that TFs in general, have lower expression levels than other proteins in *E. coli*. 

In [ ]:
Image(url = 'https://raw.githubusercontent.com/eflobau/TCD_19/master/data/protein_distro.png')

In [ ]:
from scipy.stats import ttest_ind

In [ ]:
hyp_mean_TPM = tpm_log_yome[tpm_log_yome['TF_annotation'] == 'hyp']['$log_{10}(TPM)$'].values
exp_mean_TPM = tpm_log_yome[tpm_log_yome['TF_annotation'] == 'exp']['$log_{10}(TPM)$'].values

Now, we can test if the difference of means in both distributions (hypothetical vs experimentally validated TFs) is statistically significant. 

In [ ]:
#Run one side t-test
ttest_ind(hyp_mean_TPM, exp_mean_TPM)

You can even go further and make a bootstrap test for the difference of medians.

In [ ]:
#write your code here. 

### Extending the analysis: from the transcriptome to the proteome. 

Despite not shown in their paper, the Palsson group also extended their analysis to the protein copy number levels. Quite naturally, we would expect that the differences would only amplify at the level of proteins, but let's wait and see if this difference between expression level holds. 

In [ ]:
prot.head(2)

In [ ]:
prot_yome = prot.merge(yome_genes)

In [ ]:
yome_genes.head()

In [ ]:
sns.violinplot(data = prot_yome, x = 'mean_log_val', y = 'annotation_quality',
           palette = 'Greens_r', inner = 'quartile')

#plt.savefig('../Desktop/yome_prot_cel.png', dpi = 420)

Voilà!

In [ ]:
tf_annot = []

for row in prot_yome['locus_tag']:
    if row in exp_locus_tags :
        tf_annot.append('exp')
    elif row in hyp_locus_tags:
        
        tf_annot.append('hyp')
        
    else:
        tf_annot.append('non_TF') 
        
prot_yome['TF_annotation'] = tf_annot

In [ ]:
sns.violinplot(data = prot_yome, y = 'mean_log_val', x ='TF_annotation', 
              palette = ch, inner = 'quartile');


### Challenge. 

Calculate the effect size (Cohen's d), and run bootstrap tests on this measurement for the hypothetical and experimental TFs in both the transcriptome and proteome levels. 